In [17]:
library(data.table)
library(stringr)
library(dplyr)
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0


In [2]:
#read in the metadata provided by E and use it to fill in the .fam file

Meta <- fread("~/ClownfishGWAS/data/APPC_sequencing/metadata/Larvalh2-1.csv", colClasses=c(larvalID="character"))[Send.to.NJ==1][
    , larvalID := paste0("APPC", larvalID)][ #modify sample name to match genomics IDs
    , larvalID := gsub(".", "", larvalID ,fixed = TRUE)][
    , dam := paste0("D", dam)][
    , sire := paste0("S", sire)]          

Fam <- fread("~/ClownfishGWAS/data/APPC_sequencing/DNA/filtering/imputation/max_missing90_minQ20_maf01_hwe.fam")

LibPrep <- fread("~/ClownfishGWAS/data/APPC_sequencing/metadata/APercula_gDNALIbraryPrep.csv")[, .(SampleID, Capture)][, SampleID := gsub(".", "", SampleID ,fixed = TRUE)]     

In [40]:
head(Fam)
head(Meta)
head(LibPrep)

V1,V2,V3,V4,V5,V6
Pop01_APPC1611,Pop01_APPC1611,0,0,0,-9
Pop01_APPC1917,Pop01_APPC1917,0,0,0,-9
Pop01_APPC1921,Pop01_APPC1921,0,0,0,-9
Pop01_APPC2029,Pop01_APPC2029,0,0,0,-9
Pop01_APPC2030,Pop01_APPC2030,0,0,0,-9
Pop01_APPC2101,Pop01_APPC2101,0,0,0,-9


og_larvalID,larvalID,Ucrit,SL,dam,sire,tank,date,Send.to.NJ,Group
5.06,APPC0506,8.17,4.34,D119,S162,C2:4,5/28/19,1,High
5.08,APPC0508,1.10,NA,D119,S162,C2:4,5/28/19,1,Low
5.15,APPC0515,2.27,4.32,D119,S162,C2:4,5/28/19,1,Low
5.23,APPC0523,8.33,4.32,D119,S162,C2:4,5/28/19,1,High
6.01,APPC0601,2.57,4.24,D32,S101,D1:3,5/29/19,1,Low
6.07,APPC0607,2.38,3.94,D32,S101,D1:3,5/29/19,1,Low


SampleID,Capture
APPC1611,Pop01
APPC1917,Pop01
APPC1921,Pop01
APPC2029,Pop01
APPC2030,Pop01
APPC2101,Pop01


In [3]:
#join laboratory and genetic metadata
FullMeta <- left_join(Meta, LibPrep, by=c(larvalID="SampleID"))


In [44]:
nrow(FullMeta[is.na(Capture)]) #2

[1] 2

In [46]:
nrow(FullMeta) #hmmm looks like sample IDs were written down wrong by me from the tube writing... look back at this later

[1] 156

In [4]:
FullMeta[!is.na(Capture), gen_id := paste(Capture, larvalID, sep="_")]

Warning message in `[.data.table`(FullMeta, !is.na(Capture), `:=`(gen_id, paste(Capture, :
“Invalid .internal.selfref detected and fixed by taking a (shallow) copy of the data.table so that := can add this new column by reference. At an earlier point, this data.table has been copied by R (or was created manually using structure() or similar). Avoid names<- and attr<- which in R currently (and oddly) may copy the whole data.table. Use set* syntax instead to avoid copying: ?set, ?setnames and ?setattr. If this message doesn't help, please report your use case to the data.table issue tracker so the root cause can be fixed or this message improved.”

In [6]:
head(FullMeta)
head(Fam)

og_larvalID,larvalID,Ucrit,SL,dam,sire,tank,date,Send.to.NJ,Group,Capture,gen_id
5.06,APPC0506,8.17,4.34,D119,S162,C2:4,5/28/19,1,High,Pop13,Pop13_APPC0506
5.08,APPC0508,1.10,NA,D119,S162,C2:4,5/28/19,1,Low,Pop13,Pop13_APPC0508
5.15,APPC0515,2.27,4.32,D119,S162,C2:4,5/28/19,1,Low,Pop13,Pop13_APPC0515
5.23,APPC0523,8.33,4.32,D119,S162,C2:4,5/28/19,1,High,Pop13,Pop13_APPC0523
6.01,APPC0601,2.57,4.24,D32,S101,D1:3,5/29/19,1,Low,Pop13,Pop13_APPC0601
6.07,APPC0607,2.38,3.94,D32,S101,D1:3,5/29/19,1,Low,Pop13,Pop13_APPC0607


V1,V2,V3,V4,V5,V6
Pop01_APPC1611,Pop01_APPC1611,0,0,0,-9
Pop01_APPC1917,Pop01_APPC1917,0,0,0,-9
Pop01_APPC1921,Pop01_APPC1921,0,0,0,-9
Pop01_APPC2029,Pop01_APPC2029,0,0,0,-9
Pop01_APPC2030,Pop01_APPC2030,0,0,0,-9
Pop01_APPC2101,Pop01_APPC2101,0,0,0,-9


In [58]:
(FamID[, .N, by=dam])

[1] 32

In [7]:
#the two columns we need to fill are Family ID (column 1) and the phenotype, which will be Ucrit (quant), group (case/control), or SL (quant)
#for this, we will make FamilyID the mother's ID
FamID <- left_join(FullMeta[!is.na(Capture), .(dam, gen_id)], Fam[,-"V1"], by=c(gen_id="V2"))

In [8]:
head(FamID)

dam,gen_id,V3,V4,V5,V6
D119,Pop13_APPC0506,0,0,0,-9
D119,Pop13_APPC0508,0,0,0,-9
D119,Pop13_APPC0515,0,0,0,-9
D119,Pop13_APPC0523,0,0,0,-9
D32,Pop13_APPC0601,0,0,0,-9
D32,Pop13_APPC0607,0,0,0,-9


In [11]:
#join the phenotype data
WithPhenotypeSL <- left_join(FamID[, -"V6"], FullMeta[,.(gen_id, SL)], by="gen_id")[
    is.na(SL), SL := -9]

In [19]:
fwrite(rbind(WithPhenotypeSL, Fam[V2 %!in% WithPhenotypeSL[, gen_id]], use.names=F), "~/ClownfishGWAS/data/APPC_sequencing/DNA/filtering/imputation/plink_gwas/max_missing90_minQ20_maf01_hwe.fam", col.names = FALSE, sep = "\t")

In [18]:
Fam[V2 %!in% WithPhenotypeSL[, gen_id]]

V1,V2,V3,V4,V5,V6
Pop02_APPC1010,Pop02_APPC1010,0,0,0,-9
Pop13_APPC0710,Pop13_APPC0710,0,0,0,-9
Pop14_APPC0704,Pop14_APPC0704,0,0,0,-9
Pop14_APPC0706,Pop14_APPC0706,0,0,0,-9
Pop14_APPC0707,Pop14_APPC0707,0,0,0,-9
